In [1]:
import numpy as np
import pandas
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import power_transform

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

### Define data directory and file name to load

In [2]:
day_of_week = 'Monday'

data_directory='../data/raw'
file_name=day_of_week+'-WorkingHours.pcap_ISCX.csv'

data_file=os.path.join(data_directory,file_name)

### Load the data frame

In [3]:
df=pandas.read_csv(data_file,skipinitialspace=True,low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529918 entries, 0 to 529917
Data columns (total 79 columns):
Destination Port               529918 non-null int64
Flow Duration                  529918 non-null int64
Total Fwd Packets              529918 non-null int64
Total Backward Packets         529918 non-null int64
Total Length of Fwd Packets    529918 non-null int64
Total Length of Bwd Packets    529918 non-null int64
Fwd Packet Length Max          529918 non-null int64
Fwd Packet Length Min          529918 non-null int64
Fwd Packet Length Mean         529918 non-null float64
Fwd Packet Length Std          529918 non-null float64
Bwd Packet Length Max          529918 non-null int64
Bwd Packet Length Min          529918 non-null int64
Bwd Packet Length Mean         529918 non-null float64
Bwd Packet Length Std          529918 non-null float64
Flow Bytes/s                   529854 non-null object
Flow Packets/s                 529918 non-null object
Flow IAT Mean                  

Some of the features have the character '/' in them, which might be problematic. Therefore, such features are renamed to be on the safe side.

In [4]:
df.rename(columns={'Flow Bytes/s':'Flow Bytes per sec',
                   'Flow Packets/s':'Flow Packets per sec',
                   'Fwd Packets/s':'Fwd Packets per sec',
                   'Bwd Packets/s':'Bwd Packets per sec',
                   'Fwd Avg Bytes/Bulk':'Fwd Avg Bytes per Bulk',
                   'Fwd Avg Packets/Bulk':'Fwd Avg Packets per Bulk',
                   'Bwd Avg Bytes/Bulk':'Bwd Avg Bytes per Bulk',
                   'Bwd Avg Packets/Bulk':'Bwd Avg Packets per Bulk',
                   'Down/Up Ratio':'Down Up Ratio'
                  },inplace=True)

We also quickly check the distibution of attacks in the data set.

In [5]:
df['Label'].value_counts(normalize=True)

BENIGN    1.0
Name: Label, dtype: float64

It looks like 96.90% of the row in our dataset are BENIGN communications and the remaining 3.10% are attacks on the network.

## Data Cleaning

### Remove NaN values from the data frame

In [6]:
clean_df=df.dropna()
clean_df.dtypes.value_counts()

int64      54
float64    22
object      3
dtype: int64

In [7]:
clean_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [8]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 529854 entries, 0 to 529917
Data columns (total 79 columns):
Destination Port               529854 non-null int64
Flow Duration                  529854 non-null int64
Total Fwd Packets              529854 non-null int64
Total Backward Packets         529854 non-null int64
Total Length of Fwd Packets    529854 non-null int64
Total Length of Bwd Packets    529854 non-null int64
Fwd Packet Length Max          529854 non-null int64
Fwd Packet Length Min          529854 non-null int64
Fwd Packet Length Mean         529854 non-null float64
Fwd Packet Length Std          529854 non-null float64
Bwd Packet Length Max          529854 non-null int64
Bwd Packet Length Min          529854 non-null int64
Bwd Packet Length Mean         529854 non-null float64
Bwd Packet Length Std          529854 non-null float64
Flow Bytes per sec             529854 non-null object
Flow Packets per sec           529854 non-null object
Flow IAT Mean                  

There are still two 'object' type features in the dataset: Flow Bytes per sec and Flow Packets per sec

### We now need to remove 'Infinity' values typed as string from the data.

In [9]:
index_to_drop=clean_df.index[(clean_df['Flow Packets per sec']=='Infinity')|(clean_df['Flow Bytes per sec']=='Infinity')]
print(index_to_drop)

Int64Index([    33,     77,     80,     92,     97,    103,    105,    114,
               207,    927,
            ...
            519886, 520492, 520890, 521677, 522154, 522818, 523375, 525559,
            526233, 527506],
           dtype='int64', length=373)


In [10]:
final_df=clean_df.drop(index_to_drop)

In [11]:
final_df['Flow Bytes per sec']=final_df['Flow Bytes per sec'].astype('float')
final_df['Flow Packets per sec']=final_df['Flow Packets per sec'].astype('float')

In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 529481 entries, 0 to 529917
Data columns (total 79 columns):
Destination Port               529481 non-null int64
Flow Duration                  529481 non-null int64
Total Fwd Packets              529481 non-null int64
Total Backward Packets         529481 non-null int64
Total Length of Fwd Packets    529481 non-null int64
Total Length of Bwd Packets    529481 non-null int64
Fwd Packet Length Max          529481 non-null int64
Fwd Packet Length Min          529481 non-null int64
Fwd Packet Length Mean         529481 non-null float64
Fwd Packet Length Std          529481 non-null float64
Bwd Packet Length Max          529481 non-null int64
Bwd Packet Length Min          529481 non-null int64
Bwd Packet Length Mean         529481 non-null float64
Bwd Packet Length Std          529481 non-null float64
Flow Bytes per sec             529481 non-null float64
Flow Packets per sec           529481 non-null float64
Flow IAT Mean                

Finally, we notice there are two columns called **Fwd Header Length** and **Fwd Header Length.1**. We check if they are identical and drop one of them if true.

In [13]:
if sum(final_df['Fwd Header Length']-final_df['Fwd Header Length.1'])==0:
    final_df.drop(['Fwd Header Length.1'],axis=1,inplace=True)

Now that our data frame is clean, we can pickle it into the cleaned data directory.

In [14]:
final_df.to_pickle('../data/cleaned/'+day_of_week+'_clean.pkl')

## Data Processing

Let's start by plotting histograms of the features we have in our dataset. All files are saved into the ./figures directory for later referencing.

In [16]:
features=final_df.columns.values

In [18]:
for index, feature in enumerate(features,1):
    fig = plt.figure()
    final_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/'+day_of_week+'/original_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

A cursory look at the histograms show us two things:

1. Some of the values seem to have a single spike indicating we need to look for features with a single value. Since these features carry no information, we should identify and remove them from the data frame.

2. The range of values for most of the attributes is quite large, resulting in largely skewed distributions. A log transform should be applied.

### 1.  Find the columns that have a single value and drop them from the dataframe

In [21]:
columns_to_drop=[]
for feature in features[0:-1]:  # Need to exclude the class label
    if final_df[feature].unique().size==1:
        columns_to_drop.append(feature)
columns_to_drop

['Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'CWE Flag Count',
 'Fwd Avg Bytes per Bulk',
 'Fwd Avg Packets per Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes per Bulk',
 'Bwd Avg Packets per Bulk',
 'Bwd Avg Bulk Rate']

In [22]:
reduced_df=final_df.drop(columns_to_drop,axis=1)

In [23]:
reduced_features=reduced_df.columns.values
reduced_features

array(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes per sec',
       'Flow Packets per sec', 'Flow IAT Mean', 'Flow IAT Std',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets per sec', 'Bwd Packets per sec', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count',
       'RST Flag Count', 'PS

10 features have been removed, leaving us with 78-10=68 features. 

Let's save their histograms in a separate directory. This will help us choose features for further transformations to come from the remaining feature set.

In [24]:
for index, feature in enumerate(reduced_features,1):
    fig = plt.figure()
    reduced_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/'+day_of_week+'/reduced_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

In [25]:
reduced_df.to_pickle('../data/processed/Reduced/'+day_of_week+'_processed.pkl')

### 2. Skewness transforms

The histograms also showed us that the range of values for some of the features was very large, resulting in largely skewed values. Now let's try to do a transform on the remaining features to try to reshape their distribution to a more "normal" looking form.

In [26]:
transform_candidates=np.array(['Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes per sec',
       'Flow Packets per sec', 'Flow IAT Mean', 'Flow IAT Std',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets per sec', 'Bwd Packets per sec', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance',  'Down Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size',
       'Avg Bwd Segment Size', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
       'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean',
       'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std',
       'Idle Max', 'Idle Min'])

There are 10 (one being the class label) features excluded from the transformation list. These features are binary valued, therefore transformation does not make sense.

In [27]:
no_transform_features=np.setdiff1d(reduced_features,transform_candidates)
no_transform_features

array(['ACK Flag Count', 'Destination Port', 'ECE Flag Count',
       'FIN Flag Count', 'Fwd PSH Flags', 'Label', 'PSH Flag Count',
       'RST Flag Count', 'SYN Flag Count', 'URG Flag Count'], dtype=object)

Since there are both positive and negative values across attributes as well as zero values, a log transform does not work. The standard transformer in this case is the 'Yeo-Johnson' transformation. 

Read more at: https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-transformer

In [28]:
transformed_df=pandas.DataFrame()

for feature in reduced_features:
    if feature in transform_candidates:
        transformed_df.loc[:,feature]=power_transform(reduced_df.loc[:,feature].values.reshape(-1,1),method='yeo-johnson')
    else:
        transformed_df.loc[:,feature]=reduced_df.loc[:,feature]

In [29]:
transformed_df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,529481.000000,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,...,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05,5.294810e+05
mean,10630.809682,1.803595e-17,6.999667e-16,-4.852530e-16,3.663016e-16,7.600866e-17,-3.761784e-16,6.183755e-17,-1.082157e-16,-4.251332e-16,...,-3.100466e-16,-6.969608e-16,1.537350e-16,-1.425699e-16,1.223868e-16,-2.464914e-16,1.627530e-16,-1.228163e-16,-5.754328e-17,5.883156e-17
std,21380.212038,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,0.000000,-2.336667e+00,-1.316821e+00,-1.728768e+00,-1.831608e+00,-1.393661e+00,-1.805639e+00,-1.111712e+00,-1.839440e+00,-7.152308e-01,...,-1.239444e+00,-1.070659e+01,-4.076784e-01,-3.002066e-01,-4.076444e-01,-4.075223e-01,-4.094466e-01,-3.008954e-01,-4.094466e-01,-4.094466e-01
25%,53.000000,-8.991322e-01,-2.162433e-01,-4.464220e-01,-5.771580e-01,-1.393661e+00,-8.906309e-01,-1.111712e+00,-9.450393e-01,-7.152308e-01,...,-1.239444e+00,-9.585500e-01,-4.076784e-01,-3.002066e-01,-4.076444e-01,-4.075223e-01,-4.094466e-01,-3.008954e-01,-4.094466e-01,-4.094466e-01
50%,80.000000,1.486239e-01,-2.162433e-01,1.319021e-01,8.951223e-03,2.219666e-01,-7.595747e-03,7.704517e-04,2.215489e-01,-7.152308e-01,...,4.728182e-02,8.475043e-01,-4.076784e-01,-3.002066e-01,-4.076444e-01,-4.075223e-01,-4.094466e-01,-3.008954e-01,-4.094466e-01,-4.094466e-01
75%,443.000000,6.219315e-01,6.772200e-01,4.805010e-01,4.807438e-01,5.435348e-01,3.654715e-01,1.029229e+00,4.950929e-01,1.222069e+00,...,8.858175e-01,8.475043e-01,-4.076784e-01,-3.002066e-01,-4.076444e-01,-4.075223e-01,-4.094466e-01,-3.008954e-01,-4.094466e-01,-4.094466e-01
max,65535.000000,1.705244e+00,2.075000e+00,2.885099e+00,5.345473e+00,5.087221e+00,3.616290e+00,3.435815e+00,6.975575e+00,1.694175e+00,...,2.452780e+00,3.145227e+01,2.463489e+00,3.334635e+00,2.464259e+00,2.466585e+00,2.444133e+00,3.324351e+00,2.444121e+00,2.444200e+00


We save the histograms of the transformed features once again to confirm that transformations have worked as intended.

In [30]:
for index, feature in enumerate(reduced_features,1):
    fig = plt.figure()
    transformed_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/'+day_of_week+'/transformed_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

We finally pickle the resulting data frame after processing into the processed data directory.

In [31]:
transformed_df.to_pickle('../data/processed/Normalized/'+day_of_week+'_processed.pkl')